In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys

sys.path.insert(0, '../')

from datetime import timedelta

In [2]:
from cross_signal import *
from config_paper import paper_key_id, paper_secret_key

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key ,base_url)

In [4]:
#import api2 for real trading
from config import key_id, secret_key
r_api = tradeapi.REST(key_id, secret_key)

In [5]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-06'

In [6]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        #dataframe['srsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [7]:
account = api.get_account()
print(account.status)
account.cash

ACTIVE


'25214.51'

In [8]:
raccount = r_api.get_account()
print(account.status)
raccount

ACTIVE


Account({   'account_blocked': False,
    'account_number': '010320JEWA',
    'buying_power': '138.1176',
    'cash': '-2640.8212',
    'created_at': '2020-03-20T17:46:19.948568Z',
    'currency': 'USD',
    'daytrade_count': 3,
    'daytrading_buying_power': '0',
    'equity': '2778.9388',
    'id': '176ffd14-7e0e-4957-9ad5-98306425fa04',
    'initial_margin': '2709.88',
    'last_equity': '2761.91',
    'last_maintenance_margin': '1684.43',
    'long_market_value': '5419.76',
    'maintenance_margin': '1625.928',
    'multiplier': '2',
    'pattern_day_trader': False,
    'portfolio_value': '2778.9388',
    'regt_buying_power': '138.1176',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '181.66',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [9]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [10]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [11]:
float(raccount.cash)

-2640.8212

In [12]:
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    
    #asset Manager
    assetlist=api.list_positions()
    
    for asset in assetlist:
        #if asset is in list of positions AND is more than 2.4% loss
        # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.032 :
            openPosition = api.get_position(asset.symbol)

            returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day")
            j=assetsToTrade.index(asset.symbol)
            assetsToTrade[j]= asset.symbol+' nulled for below -3.2% pnl ' +str(currenttime)
            print('Sell', asset.symbol)  
             
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
     

    #first buy conditions -------
        try:
            #set last time to buy stocks and when to start buying
            if currenttime <=1130 and currenttime >=636:    
            
            
            # Calculated trading indicators
            #populate exponential moving averages




                 # Create the buy signal here
                if data['adx'][-1] > 23 and \
                data['mfi'][-1] > 34 and \
                data['ultosc'][-1] > 32 and \
                data['rsi'][-1] < 33 and \
                data['minusdi'][-1] > data['plusdi'][-1]:
                    rbuyord=True
                    buyord=True
                    assetlist=api.list_positions()
                    rassetlist=r_api.list_positions()
                    
                    for asset in assetlist:
                        # if asset is in the list of positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            buyord=False
                            print(asset.symbol,asset.qty)
                    
                    for asset in rassetlist:
                        # if asset is in the list of real positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            rbuyord=False
                        if float(raccount.cash) < 0:
                            rbuyord=False
                            print(asset.symbol,asset.qty)
                    




                    #if asset is not already bought, then buy             
                    if buyord==True:
                        #cashBalance = account.daytrading_buying_power
                        #equity
                        cashBalance = float(account.cash)

                        targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                        returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                        print('Buy ',returned)
                        
                        
                        #real buy stocks at 1 symbol
                        try:
                            if rbuyord==True:
                                r_returned= r_api.submit_order(symbol,1,"buy","market","day")
                        except:
                            print(symbol+" failed purchase")
                    
 




    #Second buy condition after 11:30

                elif currenttime >=1135 and currenttime >=636:
                    if data['adx'][-1] > 29 and \
                    data['rsi'][-1] > 75 and \
                    data['plusdi'][-1] > 27:
                        rbuyord==True
                        buyord=True
                        assetlist=api.list_positions()
                        rassetlist=r_api.list_positions()
                        
                        
                        for asset in assetlist:
                            # if asset is in the list of positions we have already bought, then do not buy again
                            if asset.symbol == symbol:
                                buyord=False
                                print(asset.symbol,asset.qty)
                        
                        for asset in rassetlist:
                        # if asset is in the list of real positions we have already bought, then do not buy again
                            if asset.symbol == symbol:
                                rbuyord=False
                            if float(raccount.cash) < -2000:
                                rbuyord=False
                                print(asset.symbol,asset.qty)
                        
                        
                        
                        
                        if buyord==True:
                            #cashBalance = account.daytrading_buying_power
                            #equity
                            cashBalance = float(account.cash)

                            targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                            returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                            print('Buy ',returned)


                            #real buy stocks at 1 symbol
                            try:
                                if rbuyord==True:
                                    r_returned= r_api.submit_order(symbol,1,"buy","market","day")
                            except:
                                print(symbol+" failed purchase")
        except:
            print(symbol)
      
    
    
    
    
         
        
        
#Sell conditions ------
        try:
            if data['rsi'][-1] > 80:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","limit","day",data['close'][-1]) # Market order to fully close position
                print('Sell',returned)
        except:
            print(symbol)

        
    
    
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
MKTK
RTN
UTX

BRKB
BFB
Sell Order({   'asset_class': 'us_equity',
    'asset_id': '43a2bf8a-7651-4a96-872c-6ad7015b2793',
    'canceled_at': None,
    'client_order_id': '44ac51dc-b3bd-4eed-8b28-3209fc3d8edb',
    'created_at': '2020-05-06T19:10:36.503478Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'a8603911-7145-4cb0-be2d-7e8c8a8ba976',
    'legs': None,
    'limit_price': '9.97',
    'order_class': '',
    'order_type': 'limit',
    'qty': '63',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-06T19:10:36.467029Z',
    'symbol': 'CTL',
    'time_in_force': 'day',
    'type': 'limit',
    'updated_at': '2020-05-06T19:10:36.5